In [1]:
from utils import TrajectoryInformation, DummyDataLoader, TrajectoryDataPipeline
from collections import deque, defaultdict
import numpy as np

data_path = '/home/kimbring2/minerl_data/'
env_name = 'MineRLObtainDiamondDense-v0'
trajectory_name = 'v3_absolute_grape_changeling-6_37339-46767'

item_list = ['log', 'planks', 'crafting_table', 'stick', 'wooden_pickaxe', 'cobblestone', 'coal', 
             'stone_pickaxe', 'iron_ore', 'furnace', 'iron_ingot', 'iron_pickaxe', 'stone']

inventory_key_list = ['coal', 'cobblestone', 'crafting_table', 'dirt', 'furnace', 'iron_axe', 'iron_ingot', 
                      'iron_ore', 'iron_pickaxe', 'log', 'planks', 'stick', 'stone', 'stone_axe', 
                      'stone_pickaxe', 'torch', 'wooden_axe', 'wooden_pickaxe', 'pov']
data = None
for trajectory_name in TrajectoryDataPipeline.get_trajectory_names(data_path + env_name):
    print("trajectory_name: " + str(trajectory_name))
    # trajectory_name: v3_absolute_grape_changeling-6_37339-46767
    
    trajectory_information = TrajectoryInformation(data_path + env_name + "/" + trajectory_name)
    #for chain in trajectory_information.chain:
    #    print("chain.name: ", chain.name)
    
    trajectory = TrajectoryDataPipeline.load_data(data_path + env_name + "/" + trajectory_name)
    sliced_trajectory = trajectory_information.slice_trajectory_by_item(trajectory)
    
    for item in sliced_trajectory:
        if item in item_list:
            print("item: " + str(item))
            item_index = item_list.index(item)
            #print("item_index: " + str(item_index))
            
            scale = len(item_list)
            item_layer = np.zeros([64, 64, scale], dtype=np.float32)
            item_layer[:, :, item_index] = 1.0
            
            data = DummyDataLoader(data=sliced_trajectory, items_to_add=[item])

            for s, a, r, s1, d in data.batch_iter(batch_size=1, num_epochs=1, seq_len=250):
                d = np.array(d)
                d = d.astype(int)
                #print("sum(r): " + str(sum(r)))
                                
                length = (s['pov'].shape)[0]
                print("length: " + str(length))
                #print("")
                for i in range(0, length):
                    action = {key: value[i] for key, value in a.items()}
                    for key, value in action.items():
                        if key != 'camera':
                            action[key] = value
                        else:
                            #print("value[0]: " + str(value[0]))
                            #print("value[1]: " + str(value[1]))

                            camera_threshols = (abs(value[0]) + abs(value[1])) / 2.0
                            if camera_threshols > 2.5:
                                if (value[1] < 0) and ( abs(value[0]) < abs(value[1]) ):
                                    action[key] = [0, -5]
                                elif (value[1] > 0) and ( abs(value[0]) < abs(value[1]) ):
                                    action[key] = [0, 5]
                                elif (value[0] < 0) and ( abs(value[0]) > abs(value[1]) ):
                                    action[key] = [-5, 0]
                                elif (value[0] > 0) and ( abs(value[0]) > abs(value[1]) ):
                                    action[key] = [5, 0]
                                else:
                                    action[key] = [0, 0]
                            else:
                                action[key] = [0, 0]
                                
                    #print("action: " + str(action))
                    discrete_action = {key: value for key, value in action.items()}
                    #print("discrete_action: " + str(discrete_action))
                    
                    inventory_channel = np.zeros((64,64,1))
                    if 'inventory' in s:
                        region_max_height = s['pov'][i].shape[0]
                        region_max_width = s['pov'][i].shape[1]
                        rs = 8

                        if min(region_max_height, region_max_width) < rs:
                            raise ValueError("'region_size' is too large.")

                        num_element_width = region_max_width // rs

                        inventory_channel = np.zeros(shape=list(s['pov'][i].shape[:-1]) + [1], dtype=s['pov'][i].dtype)
                        for key_idx, key in enumerate(inventory_key_list):
                            item_scaled = np.clip(1 - 1 / (s['inventory'][key][i] + 1),  # Inversed
                                                          0, 1)
                            item_channel = np.ones(shape=[rs, rs, 1], dtype=s['pov'][i].dtype) * item_scaled
                            width_low = (key_idx % num_element_width) * rs
                            height_low = (key_idx // num_element_width) * rs

                            if height_low + rs > region_max_height:
                                raise ValueError("Too many elements on 'inventory'. Please decrease 'region_size' of each component.")

                            inventory_channel[height_low:(height_low + rs), width_low:(width_low + rs), :] = item_channel

                    obs = np.concatenate((s['pov'][i] / 255.0, inventory_channel), axis=2)
                    obs = np.concatenate((obs, item_layer), axis=2)
                    #print("obs.shape: " + str(obs.shape))

    #break
    print("")

/home/kimbring2/.local/lib/python3.7/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))


trajectory_name: v3_absolute_grape_changeling-17_418-2303
item: log
length: 1524

trajectory_name: v3_absolute_grape_changeling-6_37339-46767
item: log
length: 1580
item: planks
length: 174
length: 261
item: crafting_table
length: 52
length: 96
length: 168
item: stick
length: 272
item: wooden_pickaxe
length: 184
length: 235
item: cobblestone
length: 185
length: 100
item: coal
length: 131
length: 107
length: 69
item: stone_pickaxe
length: 337
item: iron_ore
length: 349
item: furnace
length: 171
length: 9
item: iron_ingot
length: 56
length: 41
length: 25
item: iron_pickaxe
length: 96
length: 108
item: stone
length: 183
length: 26
length: 74

trajectory_name: v3_absolute_grape_changeling-9_4643-34456
item: log
length: 1150
item: planks
length: 70
length: 35
item: crafting_table
length: 61
length: 108
length: 92
item: stick
length: 147
length: 201
length: 142
item: wooden_pickaxe
length: 89
item: stone
length: 30
length: 150
item: cobblestone
length: 36
length: 367
length: 139
item: stone_

KeyboardInterrupt: 

In [7]:
from utils import TrajectoryInformation, create_nodes
import numpy as np

data_path = '/home/kimbring2/minerl_data/'
env_name = 'MineRLObtainDiamondDense-v0'
trajectory_name = 'v3_absolute_grape_changeling-6_37339-46767'

trajectory_information = TrajectoryInformation(data_path + env_name + "/" + trajectory_name)
final_chain = trajectory_information.to_old_chain_format(items=trajectory_information.chain, return_time_indexes=False)

nodes = create_nodes(final_chain)
for node in nodes:
    print("node.name: ", node.name)

for i in range(0, 10):
    print("nodes[2].crafting_agent.get_crafting_action(): ", nodes[2].crafting_agent.get_crafting_action())

node.name:  log
node.name:  planks
node.name:  crafting_table
node.name:  planks
node.name:  stick
node.name:  wooden_pickaxe
node.name:  crafting_table
node.name:  dirt
node.name:  cobblestone
node.name:  coal
node.name:  dirt
node.name:  coal
node.name:  cobblestone
node.name:  coal
node.name:  stone_pickaxe
node.name:  torch
node.name:  crafting_table
node.name:  iron_ore
node.name:  furnace
node.name:  iron_ingot
node.name:  iron_pickaxe
node.name:  wooden_pickaxe
node.name:  iron_ingot
node.name:  iron_pickaxe
node.name:  iron_ingot
node.name:  furnace
node.name:  stone
node.name:  stone
node.name:  stone
nodes[2].crafting_agent.get_crafting_action():  {'place': 5}
nodes[2].crafting_agent.get_crafting_action():  {'equip': 4}
nodes[2].crafting_agent.get_crafting_action():  {'craft': 2}
nodes[2].crafting_agent.get_crafting_action():  {'nearbyCraft': 4}
nodes[2].crafting_agent.get_crafting_action():  {'nearbySmelt': 2}
nodes[2].crafting_agent.get_crafting_action():  {'place': 5}
node